# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,paranaiba,-19.6772,-51.1908,28.39,50,53,2.19,BR,1704671844
1,1,afaahiti,-17.7500,-149.2833,28.93,81,53,4.67,PF,1704671845
2,2,enewetak,11.3474,162.3373,27.11,70,7,9.47,MH,1704671846
3,3,puerto ayora,-0.7393,-90.3518,25.86,93,100,1.79,EC,1704671846
4,4,saipan,15.1355,145.7010,28.39,83,40,7.20,MP,1704671847


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display
# Configure the map plot_4
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 840,
    frame_height = 600,
    size = "Population",
    color = "City"
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
new_df = city_data_df.loc[city_data_df['Max Temp'] > 21]
new_df = new_df.loc[new_df['Max Temp'] < 27]
new_df = new_df.loc[new_df['Wind Speed'] < 4]
new_df = new_df.loc[new_df['Cloudiness'] == 0]

# Drop any rows with null values
clean_df = new_df.dropna()

# Display sample data
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,cabedelo,-6.9811,-34.8339,26.37,76,0,3.50,BR,1704671864
111,111,conde,-7.2597,-34.9075,25.70,89,0,2.57,BR,1704671918
156,156,cabo san lucas,22.8909,-109.9124,23.99,45,0,3.13,MX,1704671879
255,255,ressano garcia,-25.4420,31.9913,21.91,96,0,1.97,MZ,1704672052
297,297,malabar,-33.9667,151.2500,26.84,70,0,3.60,AU,1704672097
299,299,bokani,9.4333,5.2000,24.38,20,0,2.34,NG,1704672099
463,463,barra de santo antonio,-9.4047,-35.5072,25.60,82,0,3.71,BR,1704672255
503,503,pesqueira,-8.3578,-36.6964,24.00,53,0,3.83,BR,1704672283


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_df.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel name
30,cabedelo,BR,-6.9811,-34.8339,76,
111,conde,BR,-7.2597,-34.9075,89,
156,cabo san lucas,MX,22.8909,-109.9124,45,
255,ressano garcia,MZ,-25.4420,31.9913,96,
297,malabar,AU,-33.9667,151.2500,70,
299,bokani,NG,9.4333,5.2000,20,
463,barra de santo antonio,BR,-9.4047,-35.5072,82,
503,pesqueira,BR,-8.3578,-36.6964,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
print(geoapify_key)

63e0fbac381b4aa7bcab78f54bdd1db2


In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "limit":20,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabedelo - nearest hotel: No hotel found
conde - nearest hotel: Paladino
cabo san lucas - nearest hotel: Comfort Rooms
ressano garcia - nearest hotel: River Hill Lodge
malabar - nearest hotel: Coogee Bay Boutique Hotel
bokani - nearest hotel: No hotel found
barra de santo antonio - nearest hotel: Resort Fazenda Fiore
pesqueira - nearest hotel: Hotel Estação Cruzeiro


,City,Country,Lat,Lng,Humidity,Hotel name,Hotel Name
30,cabedelo,BR,-6.9811,-34.8339,76,,No hotel found
111,conde,BR,-7.2597,-34.9075,89,,Paladino
156,cabo san lucas,MX,22.8909,-109.9124,45,,Comfort Rooms
255,ressano garcia,MZ,-25.4420,31.9913,96,,River Hill Lodge
297,malabar,AU,-33.9667,151.2500,70,,Coogee Bay Boutique Hotel
299,bokani,NG,9.4333,5.2000,20,,No hotel found
463,barra de santo antonio,BR,-9.4047,-35.5072,82,,Resort Fazenda Fiore
503,pesqueira,BR,-8.3578,-36.6964,53,,Hotel Estação Cruzeiro


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 840,
    frame_height = 600,
    size = "Population",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)